In [7]:
from bs4 import BeautifulSoup
import urllib.request

#open site
contents = urllib.request.urlopen("https://saint-petersburg.irr.ru/real-estate/apartments-sale/").read()
soup = BeautifulSoup(contents)
mydivs = soup.findAll("div", {"class": "listing__item"})

In [8]:
#url to first pages
yandex_real_estate = urllib.request.urlopen("https://realty.yandex.ru/sankt-peterburg/kupit/kvartira/").read()
url = "https://www.avito.ru/sankt-peterburg/kvartiry?s_trg=1"
avito_real_estate = urllib.request.urlopen("https://www.avito.ru/sankt-peterburg/kvartiry/prodam").read()

In [231]:
#parse avito with beautifoul soup
avito_soup = BeautifulSoup(avito_real_estate)
url_base = "https://www.avito.ru"
flats = avito_soup.findAll("div", {"class":"item_table-description"})
page_count = 0

data_flats = parse_page(flats, [])

while True:
    try:
        page_count += 1
        print("Page: ", page_count)
        next_page = avito_soup.find("a", {"class":"pagination-page"}).get('href')
        flats = avito_soup.findAll("div", {"class":"item_table-description"})
        data_flats = parse_page(flats, data_flats)
        
        if page_count > 100:
                break
    except Error:
        break

print("finish")
print("data len: ", len(data_flats))



flats on page:  52
Flat number 0
price:
16950000

rooms:
3

area:
86.0 m^2

3-к квартира, 86 м², 7/10 эт. в Санкт-Петербурге
floor:
7, total floors: 10

adress:
 Санкт-Петербург, Пионерская улица, 50 

subway:
Чкаловская

distance to subway:
1100.0 meters

go to next page
Flat number 1
price:
3700000

rooms:
1

area:
31.3 m^2

Студия, 31.3 м², 6/11 эт. в Санкт-Петербурге
floor:
6, total floors: 11

adress:
 ул. КИМа, д. 19 лит. Ж 

subway:
Приморская

distance to subway:
1200.0 meters

go to next page
Flat number 2
price:
4890000

rooms:
3

area:
59.0 m^2

3-к квартира, 59 м², 4/5 эт. в Санкт-Петербурге
floor:
4, total floors: 5

adress:
 Санкт-Петербург, проспект Народного Ополчения, 179к1 

subway:
Проспект Ветеранов

distance to subway:
2700.0 meters

go to next page
Flat number 3
price:
2980005

rooms:
1

area:
24.6 m^2

Студия, 24.6 м², 8/19 эт. в Санкт-Петербурге
floor:
8, total floors: 19

adress:
 ул. Шувалова, стр. 2.8 

subway:
Девяткино

distance to subway:
1800.0 meters

go

1500.0 meters

go to next page
Flat number 16
price:
6150000

rooms:
2

area:
58.0 m^2

2-к квартира, 58 м², 8/25 эт. в Пушкине
floor:
8, total floors: 25

adress:
 Пушкинский район, посёлок Шушары, Окуловская улица, 7к2 

subway:
Обухово

distance to subway:
1500.0 meters

go to next page
Flat number 17
price:
4600000

rooms:
1

area:
35.4 m^2

1-к квартира, 35.4 м², 3/6 эт. в Санкт-Петербурге
floor:
3, total floors: 6

adress:
 Санкт-Петербург, Цимбалина ул. 23 

subway:
Ломоносовская

distance to subway:
1100.0 meters

go to next page
Flat number 18
price:
3850000

rooms:
2

area:
44.0 m^2

2-к квартира, 44 м², 8/9 эт. в Пушкине
floor:
8, total floors: 9

adress:
 посёлок Шушары, Пушкинский район, Санкт-Петербург, Колпинское шоссе, 34к1 

subway:
Ломоносовская

distance to subway:
1100.0 meters

go to next page
Flat number 19
price:
4808322

rooms:
1

area:
39.9 m^2

1-к квартира, 39.9 м², 4/12 эт. в Санкт-Петербурге
floor:
4, total floors: 12

adress:
 ул. Глухарская, уч. 29, стр. 

go to next page
Flat number 27
price:
6291814

rooms:
4

area:
73.1 m^2

4-к квартира, 73.1 м², 7/20 эт. в Санкт-Петербурге
floor:
7, total floors: 20

adress:
 Лагоды 

subway:
Ладожская

distance to subway:
2800.0 meters

go to next page
Flat number 28
price:
6600000

rooms:
2

area:
50.0 m^2

2-к квартира, 50 м², 7/10 эт. в Санкт-Петербурге
floor:
7, total floors: 10

adress:
 Санкт-Петербург, Долгоозёрная улица, 1 

subway:
Комендантский проспект

distance to subway:
1300.0 meters

go to next page
Flat number 29
price:
20722343

rooms:
5

area:
135.5 m^2

5-к квартира, 135.5 м², 21/25 эт. в Санкт-Петербурге
floor:
21, total floors: 25

adress:
 Комендантский 

subway:
Комендантский проспект

distance to subway:
2900.0 meters

go to next page
Flat number 30
price:
10500000

rooms:
1

area:
55.7 m^2

1-к квартира, 55.7 м², 3/8 эт. в Санкт-Петербурге
floor:
3, total floors: 8

adress:
 Смольный пр-кт, 17 

subway:
Новочеркасская

distance to subway:
1900.0 meters

go to next page
Flat


distance to subway:
1100.0 meters

go to next page
Flat number 18
price:
3850000

rooms:
2

area:
44.0 m^2

2-к квартира, 44 м², 8/9 эт. в Пушкине
floor:
8, total floors: 9

adress:
 посёлок Шушары, Пушкинский район, Санкт-Петербург, Колпинское шоссе, 34к1 

subway:
Ломоносовская

distance to subway:
1100.0 meters

go to next page
Flat number 19
price:
4808322

rooms:
1

area:
39.9 m^2

1-к квартира, 39.9 м², 4/12 эт. в Санкт-Петербурге
floor:
4, total floors: 12

adress:
 ул. Глухарская, уч. 29, стр. 1 

subway:
Комендантский проспект

distance to subway:
3800.0 meters

go to next page
Flat number 20
price:
7900000

rooms:
2

area:
53.6 m^2

2-к квартира, 53.6 м², 6/6 эт. в Санкт-Петербурге
floor:
6, total floors: 6

adress:
 Санкт-Петербург, Большой проспект Петроградской стороны, 4 

subway:
Спортивная

distance to subway:
100.0 meters

go to next page
Flat number 21
price:
3200000

rooms:
1

area:
30.0 m^2

1-к квартира, 30 м², 5/5 эт. в Санкт-Петербурге
floor:
5, total floors: 5


135.5 m^2

5-к квартира, 135.5 м², 21/25 эт. в Санкт-Петербурге
floor:
21, total floors: 25

adress:
 Комендантский 

subway:
Комендантский проспект

distance to subway:
2900.0 meters

go to next page
Flat number 30
price:
10500000

rooms:
1

area:
55.7 m^2

1-к квартира, 55.7 м², 3/8 эт. в Санкт-Петербурге
floor:
3, total floors: 8

adress:
 Смольный пр-кт, 17 

subway:
Новочеркасская

distance to subway:
1900.0 meters

go to next page
Flat number 31
price:
2616647

rooms:
1

area:
26.5 m^2

Студия, 26.5 м², 14/22 эт. в Санкт-Петербурге
floor:
14, total floors: 22

adress:
 Лагоды 

subway:
Ладожская

distance to subway:
2800.0 meters

go to next page
Flat number 32
price:
4292160

rooms:
1

area:
40.3 m^2

1-к квартира, 40.3 м², 3/17 эт. в Санкт-Петербурге
floor:
3, total floors: 17

adress:
 Коломяжский пр., 13, лит. А. 

subway:
Пионерская

distance to subway:
500.0 meters

go to next page
Flat number 33
price:
6500000

rooms:
2

area:
56.5 m^2

2-к квартира, 56.5 м², 5/5 эт. в Сан

...

97.0 m^2

3-к квартира, 97 м², 2/4 эт. в Санкт-Петербурге
floor:
2, total floors: 4

adress:
 Санкт-Петербург, Артиллерийский пер. 1 

subway:
Чернышевская

distance to subway:
700.0 meters

go to next page
Flat number 43
price:
7240000

rooms:
3

area:
87.0 m^2

3-к квартира, 87 м², 11/12 эт. в Санкт-Петербурге
floor:
11, total floors: 12

adress:
 Санкт-Петербург, Маршала Казакова ул. 68, корпус 1 

subway:
Автово

distance to subway:
4600.0 meters

go to next page
Flat number 44
price:
4900000

rooms:
3

area:
62.5 m^2

3-к квартира, 62.5 м², 7/9 эт. в Санкт-Петербурге
floor:
7, total floors: 9

adress:
 Санкт-Петербург, Энергетиков пр. 48 

subway:
Ладожская

distance to subway:
3600.0 meters

go to next page
Flat number 45
price:
5850000

rooms:
1

area:
38.0 m^2

1-к квартира, 38 м², 8/9 эт. в Санкт-Петербурге
floor:
8, total floors: 9

adress:
 Санкт-Петербург, Шевченко ул. 17 

subway:
Приморская

distance to subway:
2200.0 meters

go to next page
Flat number 46
price:
10800000


distance to subway:
1200.0 meters

go to next page
Flat number 2
price:
4890000

rooms:
3

area:
59.0 m^2

3-к квартира, 59 м², 4/5 эт. в Санкт-Петербурге
floor:
4, total floors: 5

adress:
 Санкт-Петербург, проспект Народного Ополчения, 179к1 

subway:
Проспект Ветеранов

distance to subway:
2700.0 meters

go to next page
Flat number 3
price:
2980005

rooms:
1

area:
24.6 m^2

Студия, 24.6 м², 8/19 эт. в Санкт-Петербурге
floor:
8, total floors: 19

adress:
 ул. Шувалова, стр. 2.8 

subway:
Девяткино

distance to subway:
1800.0 meters

go to next page
Flat number 4
price:
7300000

rooms:
3

area:
67.0 m^2

3-к квартира, 67 м², 12/13 эт. в Санкт-Петербурге
floor:
12, total floors: 13

adress:
 Санкт-Петербург, проспект Просвещения, 75 

subway:
Гражданский проспект

distance to subway:
1000.0 meters

go to next page
Flat number 5
price:
125000000

rooms:
5

area:
204.0 m^2

5-к квартира, 204 м², 5/5 эт. в Санкт-Петербурге
floor:
5, total floors: 5

adress:
 СПб., ул. Рубинштейна д.40/1

78.0 m^2

3-к квартира, 78 м², 7/12 эт. в Санкт-Петербурге
floor:
7, total floors: 12

adress:
 Санкт-Петербург, проспект Королёва, 48к1, подъезд 5 

subway:
Комендантский проспект

distance to subway:
1800.0 meters

go to next page
Flat number 14
price:
10300000

rooms:
5

area:
122.0 m^2

5-к квартира, 122 м², 4/5 эт. в Санкт-Петербурге
floor:
4, total floors: 5

adress:
 Санкт-Петербург, 17-я линия Васильевского острова, 70 

subway:
Василеостровская

distance to subway:
1300.0 meters

go to next page
Flat number 15
price:
8595000

rooms:
2

area:
69.7 m^2

2-к квартира, 69.7 м², 26/35 эт. в Санкт-Петербурге
floor:
26, total floors: 35

adress:
 Санкт-Петербург, проспект Обуховской Обороны, 138к2 

subway:
Обухово

distance to subway:
1500.0 meters

go to next page
Flat number 16
price:
6150000

rooms:
2

area:
58.0 m^2

2-к квартира, 58 м², 8/25 эт. в Пушкине
floor:
8, total floors: 25

adress:
 Пушкинский район, посёлок Шушары, Окуловская улица, 7к2 

subway:
Обухово

distance to 

adress:
 Санкт-Петербург, проспект Юрия Гагарина, 37, подъезд 2 

subway:
Московская

distance to subway:
800.0 meters

go to next page
Flat number 36
price:
7200000

rooms:
2

area:
64.2 m^2

2-к квартира, 64.2 м², 4/10 эт. в Санкт-Петербурге
floor:
4, total floors: 10

adress:
 Санкт-Петербург, улица Олеко Дундича, 7к1, подъезд 2 

subway:
Купчино

distance to subway:
900.0 meters

go to next page
Flat number 37
price:
6000000

rooms:
2

area:
58.1 m^2

2-к квартира, 58.1 м², 4/5 эт. в Санкт-Петербурге
floor:
4, total floors: 5

adress:
 Санкт-Петербург, Колпино, проспект Ленина, 24/8 

subway:
Купчино

distance to subway:
14500.0 meters

go to next page
Flat number 38
price:
7393459

rooms:
2

area:
68.1 m^2

2-к квартира, 68.1 м², 12/21 эт. в Санкт-Петербурге
floor:
12, total floors: 21

adress:
 Коломяжский пр., 13, лит. А. 

subway:
Пионерская

distance to subway:
500.0 meters

go to next page
Flat number 39
price:
6726360

rooms:
2

area:
63.9 m^2

2-к квартира, 63.9 м², 15/21 э

 СПб., ул. Рубинштейна д.40/11 

subway:
Достоевская

distance to subway:
200.0 meters

go to next page
Flat number 6
price:
4200000

rooms:
1

area:
37.3 m^2

1-к квартира, 37.3 м², 17/23 эт. в Санкт-Петербурге
floor:
17, total floors: 23

adress:
 посёлок Мурино, Всеволожский район, Ленинградская область, Привокзальная площадь, 1Ак1 

subway:
Девяткино

distance to subway:
100.0 meters

go to next page
Flat number 7
price:
4700000

rooms:
2

area:
45.2 m^2

2-к квартира, 45.2 м², 4/9 эт. в Санкт-Петербурге
floor:
4, total floors: 9

adress:
 Санкт-Петербург, Народная улица, 6 

subway:
Ломоносовская

distance to subway:
1100.0 meters

go to next page
Flat number 8
price:
7163640

rooms:
2

area:
60.3 m^2

2-к квартира, 60.3 м², 2/12 эт. в Санкт-Петербурге
floor:
2, total floors: 12

adress:
 ул. Глухарская, уч. 29, стр. 1 

subway:
Комендантский проспект

distance to subway:
3800.0 meters

go to next page
Flat number 9
price:
2916377

rooms:
1

area:
33.8 m^2

1-к квартира, 33.8 м², 

In [230]:
#extract all relevant data from single page
def parse_page(flats, data_flats):
    rent_count = 0
    print("flats on page: ", len(flats))
    for (index, flat) in enumerate(flats):

        print("Flat number", index)

        #price
        print("price:")
        price = price_from_json(flat)
        if(price < 200_000):
            rent_count += 1
            print("Rent found, skip")
            continue
        print(price)
        print()


        #rooms, area, floor level
        print("rooms:")
        description = flat.find("a", {"class": "item-description-title-link"})['title']
        try_rooms = description.split(",")[0][0]
        if try_rooms.isnumeric():
            rooms = int(try_rooms)
        else:
            rooms = 1
        print(rooms)
        print()

        print("area:")
        area = float(description.split(",")[1].split(" ")[1])
        print(area, "m^2")
        print()

        print(description)

        print("floor:")
        floor = description.split(",")[2].split(" ")[1].split("/")[0]
        floors_total = description.split(",")[2].split(" ")[1].split("/")[1]
        print("" + floor + ", total floors: " + floors_total)
        print()

        #subway, distance to subway, adress
        full_adress = flat.find("p", {"class": "address"}).getText()

        adress = ','.join(full_adress.split(",")[1:])
        print("adress:")
        print(adress)
        print()

        subway = full_adress.split(",")[0]
        for (ind,el) in enumerate(subway.split()):
            try:
                float(el)
                subway_station = ' '.join(subway.split()[:ind])
                distance = float(subway.split()[ind])
            except ValueError:
                pass

        print("subway:")
        print(subway_station)
        print()

        if distance < 99:
            distance *= 1000
        print("distance to subway:")
        print(distance, "meters")
        print()

        print("go to next page")
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway_station, distance))
    
    print(rent_count)
    return data_flats
    

In [59]:
#extract price
def price_from_json(flat):
    price = flat.find("div", {"class": "popup-prices"})['data-prices']
    price_ind = price.find('RUB":')
    price_cut = price[price_ind + 5:]
    price_in_rub = price_cut.split(",")[0]
    return int(price_in_rub)

In [202]:
#flat data class combining all relevant data
class Flat:
    
    def __init__(self, flat_price, flat_rooms, flat_area, flat_floor, 
                 flat_floors_total, flat_address, flat_subway_st, flat_distance_to_subway):
        self.flat_price = flat_price
        self.flat_rooms = flat_rooms 
        self.flat_area = flat_area
        self.flat_floor = flat_floor
        self.flat_floors_total = flat_floors_total
        self.flat_address = flat_address
        self.flat_subway_st = flat_subway_st
        self.flat_distance_to_subway = flat_distance_to_subway
        
    def __str__(self):
        return ''.join("adress = "+ flat_address + 
                       ", price = "+ str(flat_price)+ 
                       ", rooms = "+ str(flat_rooms)+
                        ", floor = "+ str(flat_floor)+ 
                       ", area = " + str(flat_area) +
                      ", total_floors = "+ str(flat_floors_total)+ 
                       ", subway = "+ flat_subway_st + 
                       ", distance to subway = " + flat_distance_to_subway)

In [220]:
#
subways = set()
for flat in data_flats:
    subways.add(flat[6])
print(subways)

{'Чкаловская', 'Проспект Просвещения', 'Пионерская', 'Пролетарская', 'Приморская', 'Купчино', 'Проспект Большевиков', 'Елизаровская', 'Ломоносовская', 'Комендантский проспект', 'Василеостровская', 'Ладожская', 'Девяткино', 'Спортивная', 'Московская', 'Чернышевская', 'Автово', 'Звездная', 'Обухово', 'Парнас', 'Достоевская', 'Проспект Ветеранов', 'Новочеркасская', 'Гражданский проспект'}


In [225]:
#clean up parsed data
new_data_flats = []
for flat in data_flats:
    new_flat = (flat[0],flat[1],flat[2],flat[3],flat[4],flat[5].replace('\xa0', '', 2),flat[6],flat[7])
    new_data_flats.append(new_flat)

In [229]:
#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

a = numpy.asarray(new_data_flats)
pd.DataFrame(a).to_csv("data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])